In [ ]:
!pip install --upgrade google-genai google-api-python-client google-auth-httplib2 google-auth-oauthlib

#Introduction: Building a Newsletter with a Team of AI Agents

In [ ]:
!export GOOGLE_CLOUD_PROJECT='remy-sandbox'
!export GOOGLE_CLOUD_LOCATION=global
!export GOOGLE_GENAI_USE_VERTEXAI=True

!gcloud config set project remy-sandbox

!gcloud auth application-default login --scopes=openid,https://www.googleapis.com/auth/userinfo.email,https://www.googleapis.com/auth/cloud-platform,https://www.googleapis.com/auth/drive.readonly

In [ ]:
!export GOOGLE_CLOUD_PROJECT='remy-sandbox'
!export GOOGLE_CLOUD_LOCATION=global
!export GOOGLE_GENAI_USE_VERTEXAI=True


client = genai.Client(vertexai=True, project='remy-sandbox', location='us-central1', http_options=HttpOptions(api_version="v1"))
MODEL_ID = "gemini-2.5-flash"

def model_response(text,model_id):
   response = client.models.generate_content(
    model=genai.GenerativeModel(model_id),
    contents = text
)
   return response.text

In [ ]:
from google import genai
import json
from IPython.display import display, HTML, Markdown
from google.genai.types import HttpOptions


def show_json(obj):
  print(json.dumps(obj.model_dump(exclude_none=True), indent=2))
  return json.dumps(obj.model_dump(exclude_none=True), indent=2)

def show_parts(r):
  parts = r.candidates[0].content.parts
  if parts is None:
    finish_reason = r.candidates[0].finish_reason
    print(f'{finish_reason=}')
    return
  for part in r.candidates[0].content.parts:
    if part.text:
      display(Markdown(part.text))
      output = part.text
    elif part.executable_code:
      display(Markdown(f'```python\n{part.executable_code.code}\n```'))
      output = part.executable_code
    else:
      show_json(part)

  grounding_metadata = r.candidates[0].grounding_metadata
  if grounding_metadata and grounding_metadata.search_entry_point:
    display(HTML(grounding_metadata.search_entry_point.rendered_content))
  return output

!export GOOGLE_CLOUD_PROJECT='remy-sandbox'
!export GOOGLE_CLOUD_LOCATION=global
!export GOOGLE_GENAI_USE_VERTEXAI=True




#client = genai.Client(api_key="")

client = genai.Client(vertexai=True, project='remy-sandbox', location='us-central1', http_options=HttpOptions(api_version="v1"))
MODEL_ID = "gemini-2.5-flash"

def model_response(text,model_id):
   response = client.models.generate_content(
    model=model_id,
    contents = text
)
   return response.text



##Creating the search agent

In [ ]:
search_tool = {'google_search': {}}
search_chat = client.chats.create(model="gemini-2.5-flash", config={'tools': [search_tool]})

#Create the Google Drive Agent

In [ ]:
import os
import json
from dataclasses import dataclass, asdict
from typing import List, Optional
from datetime import datetime
from dateutil import parser as dateparser

from googleapiclient.discovery import build
from google.auth import default
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials

In [ ]:
# gemini_gdrive_tool.py

from __future__ import annotations

import json
import os
from typing import Any, Dict, List, Optional

from google import genai
from google.genai import types
from google.auth import default
from googleapiclient.discovery import build
from google.genai.types import HttpOptions


# --- Configure the Gemini client ---
client = genai.Client(vertexai=True, project='remy-sandbox', location='us-central1', http_options=HttpOptions(api_version="v1"))
MODEL_ID = "gemini-2.5-flash"


# --- Tool Implementation: Google Drive search ---
def search_google_drive(
    query: str,
    max_results: int = 10,
) -> Dict[str, Any]:
    """
    Execute a Google Drive search and return a JSON-serializable payload.
    Query string is used to search for file names.
    """
    creds, _ = default()
    drive_service = build('drive', 'v3', credentials=creds)

    results = drive_service.files().list(
        q=f"name contains '{query}' and mimeType != 'application/vnd.google-apps.folder'",
        pageSize=max_results,
        fields="nextPageToken, files(id, name, mimeType, webViewLink)"
    ).execute()

    items = results.get('files', [])
    
    files_content = []
    for item in items:
        file_id = item['id']
        file_name = item['name']
        mime_type = item['mimeType']
        
        content = None
        try:
            if mime_type == 'application/vnd.google-apps.document': # Google Doc
                request = drive_service.files().export_media(fileId=file_id, mimeType='text/plain')
                content = request.execute().decode('utf-8')
            elif mime_type.startswith('text/') or mime_type == 'application/json' or mime_type == 'application/rtf' or mime_type == 'application/pdf':
                 if mime_type == 'application/pdf':
                     # can't just read pdfs, would need a library. Skipping for now.
                     continue
                 request = drive_service.files().get_media(fileId=file_id)
                 content = request.execute().decode('utf-8')
        except Exception as e:
            print(f'Error reading file {file_name}: {e}')
            content = f'Error reading file: {e}'

        if content:
            files_content.append({
                "name": file_name,
                "summary": content[:800], # Truncate for brevity
                "link": item['webViewLink']
            })

    return {
        "query": query,
        "count": len(files_content),
        "results": files_content,
    }

# --- Tool declaration (matches Gemini function-calling schema) ---
gdrive_tool = types.Tool(
    function_declarations=[
        {
            "name": "search_google_drive",
            "description": (
                "Search Google Drive for documents with a given query in their name. "
                "Returns a JSON list of documents with metadata and content."
            ),
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "Query string to search for in file names in Google Drive.",
                    },
                    "max_results": {
                        "type": "integer",
                        "minimum": 1,
                        "maximum": 50,
                        "default": 10,
                        "description": "Maximum number of files to return (cap at 50). ignited by users ask",
                    },
                },
                "required": ["query"],
            },
        }
    ]
)

# --- Dispatch: map function calls to Python implementations ---
def handle_tool_call(name: str, args: Dict[str, Any]) -> Dict[str, Any]:
    if name == "search_google_drive":
        return search_google_drive(**args)
    raise ValueError(f"Unknown tool: {name}")

config = types.GenerateContentConfig(tools=[gdrive_tool])
# --- Conversation with function calling loop ---
def ask_gemini_with_gdrive(prompt: str, model: str = "gemini-2.5-flash") -> str:
    contents = [
        types.Content(role="user", parts=[ types.Part(text=prompt) ])
    ]

    response = client.models.generate_content(
        model=model,
        contents=contents,
        config=config,
    )

    tool_call = None
    for cand in response.candidates or []:
        for part in (cand.content.parts or []):
            if getattr(part, "function_call", None):
                tool_call = part.function_call
                break
        if tool_call:
            break

    if not tool_call:
        return response.text

    args = tool_call.args
    tool_result = handle_tool_call(tool_call.name, args)

    function_response_part = types.Part.from_function_response(
        name=tool_call.name,
        response={"result": tool_result},
    )

    contents.append(response.candidates[0].content)
    contents.append(types.Content(role="user", parts=[function_response_part]))

    final = client.models.generate_content(
        model=model,
        contents=contents,
        config=config,
    )

    return final.text

# --- Example usage ---
if __name__ == "__main__":
    demo_prompt = (
        "Find 5 of my stories in my Google Drive about 'AI'. Summarize them."
    )
    print(ask_gemini_with_gdrive(demo_prompt))


#Build an autonomous system for generating newsletters

In [ ]:
problem = """You are the curator for the Agentville newsletter. Your task is to assemble the content for the centennial edition, covering the developments in autonomous agents. The newsletter should be in a witty format
and should cover different dimensions of autonomous agents."""

##Planner in action

In [ ]:
plan = model_response(f'''You are a planning agent inside an autonomous multi-agent system.\nYour job is to take a user's goal:{problem} and break it into a structured to-do list of clear steps.\nYou have access to Search and Google Drive retrieval tools.\n\nInstructions:\n1. Understand the user’s goal.\n2. Break it down into the smallest actionable steps needed to achieve it. You are not supposed to come up with\nthe final answer to the problem.\n3. Each step must be atomic (can be completed by a single specialized agent).\n4. Order the steps logically. Outline which agent among search and gdrive needs to be used to tackle each step.\n5.Format of the step: Tool name, step description. Available tool names are agent_search and agent_gdrive\n6.Separate each step with a --\n7. Include a final step to "summarize the overall findings" once all tasks are done.''',"gemini-2.5-pro")

In [ ]:
print (plan)

In [ ]:
plan_steps = plan.split('--')

## Helper functions to call Explorer and Scholar

In [ ]:
def handle_search(memory, step: str):
    print(f"[SEARCH] Handling search step: {step}")
    r = search_chat.send_message(f'''For the step: {step}, extract relevant context from memory: {memory} and execute the step.''')
    response = show_parts(r)
    return response

def handle_gdrive(memory, step: str):
    print(f"[GDRIVE] Handling gdrive step: {step}")
    response = ask_gemini_with_gdrive(f'''Find out documents from google drive corresponding to the {step}. Use {memory} for context.''')
    return response

def handle_default(memory, step: str):
    print(f"[DEFAULT] Handling generic step: {step}")
    response = model_response(f'''For the step: {step}, extract relevant context from memory: {memory} and execute the step.''',"gemini-2.5-flash")
    return response


##Agent orchestration

In [ ]:
memory = {}
for i in range(1,len(plan_steps)+1):
  print (f"{plan_steps[i-1]}")
  if "agent_search" in plan_steps[i-1]:
        agent_response = handle_search(memory, plan_steps[i-1])
  elif "agent_gdrive" in plan_steps[i-1]:
        agent_response = handle_gdrive(memory, plan_steps[i-1])
  else:
        agent_response = handle_default(memory, plan_steps[i-1])
  memory[plan_steps[i-1]] = agent_response
  print (memory)

In [ ]:
print (memory)

In [ ]:
print (model_response(f'''Render the response:{memory} in the form of a detailed report in markdown format with proper indentation.
Preserve the hyperlinks to the research papers and links cited.''','gemini-2.5-flash'))

## Critic

In [ ]:
critic_response = model_response(f'''For the given problem:{problem}, a bunch of agents worked together to curate the response: {memory}. Your
job is to analyze how well the original plan: {plan} was executed and suggest improvements for the overall system and individual agents involved.''',"gemini-2.5-pro")

In [ ]:
print (critic_response)

# Bringing everything together: Autonomous Agentic Module

In [ ]:
def autonomous_newsletter_generation(problem):
  plan = model_response(f'''You are a planning agent inside an autonomous multi-agent system.\nYour job is to take a user's goal:{problem} and break it into a structured to-do list of clear steps.\nYou have access to Search and Google Drive retrieval tools.\n\nInstructions:\n1. Understand the user’s goal.\n2. Break it down into the smallest actionable steps needed to achieve it. You are not supposed to come up with\nthe final answer to the problem.\n3. Each step must be atomic (can be completed by a single specialized agent).\n4. Order the steps logically. Outline which agent among search and gdrive needs to be used to tackle each step.\n5.Format of the step: Tool name, step description. Available tool names are agent_search and agent_gdrive\n6.Separate each step with a --\n7. Include a final step to "summarize the overall findings" once all tasks are done.''',"gemini-2.5-pro")
  print ("Plan is",plan)
  plan_steps = plan.split('--')
  memory = {}
  for i in range(1,len(plan_steps)+1):
     print (f"{plan_steps[i-1]}")
     if "agent_search" in plan_steps[i-1]:
        agent_response = handle_search(memory, plan_steps[i-1])
     elif "agent_gdrive" in plan_steps[i-1]:
        agent_response = handle_gdrive(memory, plan_steps[i-1])
     else:
        agent_response = handle_default(memory, plan_steps[i-1])
     memory[plan_steps[i-1]] = agent_response

  agent_report = model_response(f'''Render the response:{memory} in the form of a detailed report in markdown format with proper indentation.\nPreserve the hyperlinks to the documents and links cited.''','gemini-2.5-flash')
  critic_response = model_response(f'''For the given problem:{problem}, a bunch of agents worked together to curate the response: {memory}. Your\njob is to analyze how well the original plan: {plan} was executed and suggest improvements for the overall system and individual agents involved.''',"gemini-2.5-pro")
  return agent_report, critic_response


In [ ]:
report, analysis = autonomous_newsletter_generation('''Generate a newsletter around memory techniques used for
autonomous agentic systems''')